# Creación Dimensión Productos

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col, current_timestamp

# Lectura directa desde Silver
df_silver_prod = spark.read.table("workspace.silver.productos").dropDuplicates(["codigo_producto"])

window_spec = Window.orderBy("codigo_producto")

dim_productos = df_silver_prod.withColumn("producto_key", row_number().over(window_spec)) \
    .select(
        col("producto_key"),
        col("codigo_producto").alias("codigo_producto_bk"),
        col("nombre"),
        col("categoria"),
        col("unidad_de_medida"),
        col("precio_con_iva"),
        col("costo_sin_iva"),
        current_timestamp().alias("fecha_carga")
    )

# Guardar en Gold
dim_productos.write.mode("overwrite").saveAsTable("workspace.gold.dim_productos")
print("✅ dim_productos creada.")